In [56]:
import numpy as np

In [57]:
from lightfm.datasets import fetch_movielens

In [58]:
from lightfm import LightFM

In [59]:
# fetch data and format it
from lightfm.data import Dataset
dataset = Dataset(False, False)
users = [1,2,3]
items = ['1','2','3']
dataset.fit(users, items)
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 3, num_items 3.


In [60]:
#print training and testing data
print(repr(dataset))

In [61]:
#create model
model = LightFM(loss='warp')

In [63]:
#train model
features = [1, 2, 3]
dataset.fit_partial(items=items, item_features=(features))
interactions, weights = dataset.build_interactions((users, items))
print(repr(interactions))

ValueError: Item id 2 not in item id mapping. Make sure you call the fit method.

In [48]:
item_features = dataset.build_item_features([['3', '1'], ['2', '2'], ['3', '3']])
print(repr(item_features))

ValueError: item id 3 not in item id mappings.

In [17]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))
        
        top_items = data['item_labels'][np.argsort(-scores)]
        
        print("User %s" % user_id)
        print("    Known positivies:")
        
        for x in known_positives[:3]:
            print("      %s" % x)
        
        print("    recommended:")
        
        for x in top_items[:3]:
            print("          %s" % x)

In [15]:
sample_recommendation(model, data, [3, 25, 4])

User 3
    Known positivies:
      Seven (Se7en) (1995)
      Contact (1997)
      Starship Troopers (1997)
    recommended:
          Scream (1996)
          Air Force One (1997)
          Game, The (1997)
User 25
    Known positivies:
      Dead Man Walking (1995)
      Star Wars (1977)
      Fargo (1996)
    recommended:
          Fargo (1996)
          Contact (1997)
          English Patient, The (1996)
User 4
    Known positivies:
      Rumble in the Bronx (1995)
      Batman Forever (1995)
      To Wong Foo, Thanks for Everything! Julie Newmar (1995)
    recommended:
          Raiders of the Lost Ark (1981)
          Alien (1979)
          Blade Runner (1982)


In [16]:
print(data)

{'train': <943x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 49906 stored elements in COOrdinate format>, 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>, 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 1682 stored elements in Compressed Sparse Row format>, 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object), 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}


In [19]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

train_precision = precision_at_k(model, data['train'], k=10).mean()
test_precision = precision_at_k(model, data['test'], k=10).mean()

train_auc = auc_score(model, data['train']).mean()
test_auc = auc_score(model, data['test']).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.52, test 0.08.
AUC: train 0.96, test 0.92.
